In [3]:
import numpy as np
import pandas as pd 
import logging

from gensim.models import Word2Vec


from words_parser import ingredient_parser

In [7]:
 
data = pd.read_csv(r'C:\xampp\htdocs\3161Database files\recipe_recommender\app\csvfiles\parsed_recipesv4.csv')
data['parsed_ingredients'] = data.RecipeIngredientParts.apply(ingredient_parser)
data.head()

,RecipeId,Name,CookTime,RecipeCategory,Keywords,RecipeIngredientParts,Calories,FatContent,CholesterolContent,SodiumContent,SugarContent,ProteinContent,RecipeInstructions,parsed_ingredients
0,38,Low-Fat Berry Blue Frozen Dessert,PT24H,Frozen Desserts,"['Dessert','Low Protein','Low Cholesterol','He...","['blueberries','granulated sugar','vanilla yog...",170.9,2.5,8.0,29.8,30.2,3.2,"[Toss 2 cups berries with sugar., Let stand fo...","[blueberry, granulated sugar, vanilla yogurt, ..."
1,39,Biryani,PT25M,Chicken Breast,"['Chicken Thigh & Leg','Chicken','Poultry','Me...","['saffron','milk','hot green chili peppers','o...",1110.7,58.8,372.8,368.4,20.4,63.4,[Soak saffron in warm milk for 5 minutes and p...,"[saffron, milk, chili, onion, clove, peppercor..."
2,40,Best Lemonade,PT5M,Beverages,"['Low Protein','Low Cholesterol','Healthy','Su...","['sugar','lemons','rind of','lemon','zest of',...",311.1,0.2,0.0,1.8,77.2,0.3,"[Into a 1 quart Jar with tight fitting lid, pu...","[sugar, lemon, lemon, lemon juice]"
3,41,Carina's Tofu-Vegetable Kebabs,PT20M,Soy/Tofu,"['Beans','Vegetable','Low Cholesterol','Weekni...","['extra firm tofu','eggplant','zucchini','mush...",536.1,24.0,0.0,1558.6,32.1,29.3,"[Drain the tofu, carefully squeezing out exces...","[tofu, eggplant, zucchini, mushroom, soy sauce..."
4,42,Cabbage Soup,PT30M,Vegetable,"['Low Protein','Vegan','Low Cholesterol','Heal...","['plain tomato juice','cabbage','onion','carro...",103.6,0.4,0.0,959.3,17.7,4.3,"[Mix everything together and bring to a boil.,...","[tomato juice, cabbage, onion, carrot, celery]"


In [75]:
# get corpus with the documents sorted in alphabetical order
def get_and_sort_ingr(data):
    ingreds_sorted = []
    for doc in data.parsed_ingredients.values:
        ingreds_sorted.append(doc)
    return ingreds_sorted

ingreds = get_and_sort_ingr(data[:5000])
#for i in ingreds:
#    print(i)
print(f"Length of ingreds: {len(ingreds)}")

Length of ingreds: 5000


In [77]:
thirdofcorpus = round(len(ingreds)*.25)
#print(len(ingreds))
print(len(ingreds[:thirdofcorpus]))

lengths = [len(doc) for doc in ingreds]
print(round(sum(lengths)/len(lengths)))
avg_len = float(sum(lengths)) / len(lengths)
avg_len


1250
6


6.3038

In [80]:
# train word2vec model 
sg = 0 # CBOW: build a language model that correctly predicts the center word given the context words in which the center word appears
workers = 8 # number of CPUs
window = 6 # window size: average length of each document 
min_count = 1 # unique ingredients are important to decide recipes 

model_cbow = Word2Vec(ingreds, sg=sg, workers=workers, window=window, min_count=min_count, vector_size=100)

In [81]:
#Summarize the loaded model
print(model_cbow)

#Summarize vocabulary
words = list(model_cbow.wv.index_to_key)
words.sort()
print(words)

#Acess vector for one word
print("check ",model_cbow.wv['tomato'])

# most similar
model_cbow.wv.most_similar(u'tomato')
#print("after stock")
#model_cbow.wv.similarity('cauliflower', 'cauliflower just larger than potato')

Word2Vec<vocab=1239, vector_size=100, alpha=0.025>
['accent', 'achar', 'acinus di pepe pasta', 'acorn squash', 'adobo sauce', 'adzuki bean', 'albacore tuna', 'ale', 'alfredo sauce', 'allspice', 'allspice berry', 'alphabet pasta', 'alum', 'amaretti cookie', 'amaretto', 'amaretto liqueur', 'american cheese', 'american cheese spread', 'amish friendship starter', 'anaheim chili', 'anaheim chilies', 'ancho chile', 'ancho chilies', 'andouille sausage', 'angel coconut', 'angel hair pasta', 'apple', 'apple butter', 'apple pie', 'apple vinegar', 'applesauce', 'apricot', 'apricot gelatin', 'apricot puree', 'arbol chile', 'arborio rice', 'arrowroot', 'artichoke', 'artichoke heart', 'artificial sweetener', 'artificial vanilla flavoring', 'arugula', 'asiago cheese', 'asparagus', 'asparagus spear', 'atlantic salmon fillet', 'aubergine', 'aunt janes krazy up', 'avocado', 'avocado dip', 'back bacon', 'bacon', 'bacon bit', 'bacon dripping', 'bacon grease', 'bagel', 'baguette', 'baked ham', 'baker choco

[('bell', 0.99931800365448),
 ('tomato sauce', 0.9991217255592346),
 ('celery', 0.9990849494934082),
 ('beef', 0.99894779920578),
 ('parsley', 0.9988266229629517),
 ('tomato paste', 0.9986143708229065),
 ('carrot', 0.9984724521636963),
 ('rice', 0.9984385371208191),
 ('mushroom', 0.9984091520309448),
 ('basil', 0.9982789158821106)]

In [82]:
model_cbow.save('models/model_cbow_ingredients.bin')

In [83]:
class MeanEmbeddingVectorizer(object):

	def __init__(self, word_model):
		self.word_model = word_model
		self.vector_size = word_model.wv.vector_size

	def fit(self):  # comply with scikit-learn transformer requirement
		return self

	def transform(self, docs):  # comply with scikit-learn transformer requirement
		doc_word_vector = self.word_average_list(docs)
		return doc_word_vector

	def word_average(self, sent):
		"""
		Compute average word vector for a single doc/sentence.
		:param sent: list of sentence tokens
		:return:
			mean: float of averaging word vectors
		"""
		mean = []
		for word in sent:
			if word in self.word_model.wv.index_to_key:
				mean.append(self.word_model.wv.get_vector(word))

		if not mean:  # empty words
			# If a text is empty, return a vector of zeros.
			logging.warning("cannot compute average owing to no vector for {}".format(sent))
			return np.zeros(self.vector_size)
		else:
			mean = np.array(mean).mean(axis=0)
			return mean


	def word_average_list(self, docs):
		"""
		Compute average word vector for multiple docs, where docs had been tokenized.
		:param docs: list of sentence in list of separated tokens
		:return:
			array of average word vector in shape (len(docs),)
		"""
		return np.vstack([self.word_average(sent) for sent in docs])

In [84]:
# encode document by averaging word embeddings

# load model 
loaded_model = Word2Vec.load('models/model_cbow_ingredients.bin')
if loaded_model:
    print("Successfully loaded model")

mean_vec_tr = MeanEmbeddingVectorizer(loaded_model)
doc_vec = mean_vec_tr.transform(ingreds)

Successfully loaded model
cannot compute average owing to no vector for []
cannot compute average owing to no vector for []
cannot compute average owing to no vector for []
